<a href="https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/beta/AlphaFold2_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AlphaFold2 w/ MMseqs2 Batch

Easy to use version of AlphaFold 2 [(Jumper et al. 2021, Nature)](https://www.nature.com/articles/s41586-021-03819-2) a protein structure prediction pipeline, with an API hosted at the Södinglab based on the MMseqs2 server [(Mirdita et al. 2019, Bioinformatics)](https://academic.oup.com/bioinformatics/article/35/16/2856/5280135) for the multiple sequence alignment creation. 

**Usage**

input_dir: directory with only fasta files stored in Google Drive

result_dir: results will be written to the result directory in Google Drive

<strong>For detailed instructions, see <a href="#Instructions">bottom</a> of notebook!</strong>




In [ ]:
#@title Mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [2]:
#@title Input protein sequence, then hit `Runtime` -> `Run all`
from google.colab import files
import os
import os.path
import re
import hashlib

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

input_dir = '/content/drive/MyDrive/input_fasta' #@param {type:"string"}

result_dir = '/content/drive/MyDrive/result' #@param {type:"string"}

# number of models to use
#@markdown ---
#@markdown ### Advanced settings
msa_mode = "MMseqs2 (UniRef+Environmental)" #@param ["MMseqs2 (UniRef+Environmental)", "MMseqs2 (UniRef only)","single_sequence","custom"]
num_models = 5 #@param [1,2,3,4,5] {type:"raw"}
use_msa = True if msa_mode.startswith("MMseqs2") else False
use_env = True if msa_mode == "MMseqs2 (UniRef+Environmental)" else False
use_custom_msa = False
use_amber = False #@param {type:"boolean"}
use_templates = False 
use_turbo = True
use_ptm = True
max_recycles = 3
tol = 0
is_training = False 
num_samples = 1 
num_ensemble = 1 
show_images = False
save_tmp_pdb = True
rank_by = "pLDDT" 
save_pae_json = False

homooligomer =  "1" 
homooligomer = re.sub("[:/]+",":",homooligomer)
if len(homooligomer) == 0: homooligomer = "1"
homooligomer = re.sub("[^0-9:]", "", homooligomer)
homooligomers = [int(h) for h in homooligomer.split(":")]
max_msa = "512:1024"

max_msa_clusters, max_extra_msa = [int(x) for x in max_msa.split(":")]
#@markdown Don't forget to hit `Runtime` -> `Run all` after updating form


with open(f"run.log", "w") as text_file:
    text_file.write("num_models=%s\n" % num_models)
    text_file.write("use_amber=%s\n" % use_amber)
    text_file.write("use_msa=%s\n" % use_msa)
    text_file.write("msa_mode=%s\n" % msa_mode)
    text_file.write("use_templates=%s\n" % use_templates)
    text_file.write("homooligomer=%s\n" % homooligomer)


In [ ]:


#@title Install software
#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left.
use_amber_relax = use_amber 

import os
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

import jax
if jax.local_devices()[0].platform == 'tpu':
  raise RuntimeError('Colab TPU runtime not supported. Change it to GPU via Runtime -> Change Runtime Type -> Hardware accelerator -> GPU.')
elif jax.local_devices()[0].platform == 'cpu':
  raise RuntimeError('Colab CPU runtime not supported. Change it to GPU via Runtime -> Change Runtime Type -> Hardware accelerator -> GPU.')

from IPython.utils import io
import subprocess
import tqdm.notebook

GIT_REPO = 'https://github.com/deepmind/alphafold'
SOURCE_URL = 'https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar'
PARAMS_DIR = './alphafold/data/params'
PARAMS_PATH = os.path.join(PARAMS_DIR, os.path.basename(SOURCE_URL))
TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

# if not already installed
try:
  total = 100 if use_amber_relax else 55
  with tqdm.notebook.tqdm(total=total, bar_format=TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:
      if not os.path.isdir("alphafold"):
        %shell rm -rf alphafold
        %shell git clone {GIT_REPO} alphafold
        %shell (cd alphafold; git checkout 1e216f93f06aa04aa699562f504db1d02c3b704c --quiet)

        # colabfold patches
        %shell mkdir --parents tmp
        %shell wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/main/beta/colabfold.py
        %shell wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/main/beta/pairmsa.py
        %shell wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/main/beta/protein.patch -P tmp/
        %shell wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/main/beta/config.patch -P tmp/
        %shell wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/main/beta/model.patch -P tmp/
        %shell wget -qnc https://raw.githubusercontent.com/sokrypton/ColabFold/main/beta/modules.patch -P tmp/

        # hhsuite + reformat.pl
        %shell curl -fsSL https://github.com/soedinglab/hh-suite/releases/download/v3.3.0/hhsuite-3.3.0-SSE2-Linux.tar.gz | tar xz -C tmp/

        # Apply multi-chain patch from Lim Heo @huhlim
        %shell patch -u alphafold/alphafold/common/protein.py -i tmp/protein.patch
        
        # Apply patch to dynamically control number of recycles (idea from Ryan Kibler)
        %shell patch -u alphafold/alphafold/model/model.py -i tmp/model.patch
        %shell patch -u alphafold/alphafold/model/modules.py -i tmp/modules.patch
        %shell patch -u alphafold/alphafold/model/config.py -i tmp/config.patch
        pbar.update(4)

        %shell pip3 install ./alphafold
        pbar.update(5)
      
        # speedup from kaczmarj
        %shell mkdir --parents "{PARAMS_DIR}"
        %shell curl -fsSL "{SOURCE_URL}" | tar x -C "{PARAMS_DIR}"
        pbar.update(14+27)

        #######################################################################
        %shell sudo apt install --quiet --yes hmmer
        pbar.update(3)

        # Install py3dmol.
        %shell pip install py3dmol
        pbar.update(1)

        # Create a ramdisk to store a database chunk to make Jackhmmer run fast.
        %shell sudo mkdir -m 777 --parents /tmp/ramdisk
        %shell sudo mount -t tmpfs -o size=9G ramdisk /tmp/ramdisk
        pbar.update(1)
      else:
        pbar.update(55)

      if use_amber_relax:
        if not os.path.isfile("stereo_chemical_props.txt"):
          # Install OpenMM and pdbfixer.
          %shell rm -rf /opt/conda
          %shell wget -q -P /tmp \
            https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh \
              && bash /tmp/Miniconda3-latest-Linux-x86_64.sh -b -p /opt/conda \
              && rm /tmp/Miniconda3-latest-Linux-x86_64.sh
          pbar.update(4)

          PATH=%env PATH
          %env PATH=/opt/conda/bin:{PATH}
          %shell conda update -qy conda \
              && conda install -qy -c conda-forge \
                python=3.7 \
                openmm=7.5.1 \
                pdbfixer
          pbar.update(40)

          %shell wget -q -P /content \
            https://git.scicore.unibas.ch/schwede/openstructure/-/raw/7102c63615b64735c4941278d92b554ec94415f8/modules/mol/alg/src/stereo_chemical_props.txt
          pbar.update(1)
          %shell mkdir -p /content/alphafold/common
          %shell cp -f /content/stereo_chemical_props.txt /content/alphafold/common

          # Apply OpenMM patch.
          %shell pushd /opt/conda/lib/python3.7/site-packages/ && \
              patch -p0 < /content/alphafold/docker/openmm.patch && \
              popd
        else:
          pbar.update(45)

except subprocess.CalledProcessError:
  print(captured)
  raise

########################################################################################
# --- Python imports ---
import colabfold as cf
import pairmsa
import sys
import pickle
if use_amber_relax:
  sys.path.append('/opt/conda/lib/python3.7/site-packages')

if "/content/tmp/bin" not in os.environ['PATH']:
  os.environ['PATH'] += ":/content/tmp/bin:/content/tmp/scripts"


from urllib import request
from concurrent import futures
from google.colab import files
import json
from matplotlib import gridspec
import matplotlib.pyplot as plt
import numpy as np
import py3Dmol

from alphafold.model import model
from alphafold.model import config
from alphafold.model import data

from alphafold.data import parsers
from alphafold.data import pipeline
from alphafold.data.tools import jackhmmer

from alphafold.common import protein

if use_amber_relax:
  from alphafold.relax import relax
  from alphafold.relax import utils




In [ ]:
#@title Gather input features, predict structure

def _placeholder_template_feats(num_templates_, num_res_):
  return {
      'template_aatype': np.zeros([num_templates_, num_res_, 22], np.float32),
      'template_all_atom_masks': np.zeros([num_templates_, num_res_, 37, 3], np.float32),
      'template_all_atom_positions': np.zeros([num_templates_, num_res_, 37], np.float32),
      'template_domain_names': np.zeros([num_templates_], np.float32),
      'template_sum_probs': np.zeros([num_templates_], np.float32),
  }

def parse_results(prediction_result, processed_feature_dict):
  b_factors = prediction_result['plddt'][:,None] * prediction_result['structure_module']['final_atom_mask']
  out = {"unrelaxed_protein": protein.from_prediction(processed_feature_dict, prediction_result, b_factors=b_factors),
         "plddt": prediction_result['plddt'],
         "pLDDT": prediction_result['plddt'].mean(),
         "dists": prediction_result["distogram"]["bin_edges"][prediction_result["distogram"]["logits"].argmax(-1)],
         "adj": jax.nn.softmax(prediction_result["distogram"]["logits"])[:,:,prediction_result["distogram"]["bin_edges"] < 8].sum(-1)}
  if "ptm" in prediction_result:
    out.update({"pae": prediction_result['predicted_aligned_error'],
                "pTMscore": prediction_result['ptm']})
  return out

model_names = ['model_1', 'model_2', 'model_3', 'model_4', 'model_5'][:num_models]
total = len(model_names) * num_samples
if use_amber_relax:
  if relax_all: total += total
  else: total += 1


### run
for filename in os.listdir(input_dir):
  jobname=os.path.splitext(filename)[0]
  filepath = input_dir+"/"+filename
  print("Running: "+filepath)
  with open(filepath) as f:
    input_fasta_str = f.read()
  (seqs, header) = pipeline.parsers.parse_fasta(input_fasta_str)
  seq = seqs[0]
  if os.path.isfile(result_dir+"/"+jobname+".result.zip"):
    continue

  # prediction directory
  output_dir = 'prediction_' + cf.get_hash(seq)[:5]
  os.makedirs(output_dir, exist_ok=True)
  # delete existing files in working directory
  for f in os.listdir(output_dir):
    os.remove(os.path.join(output_dir, f))

  prefix = cf.get_hash("".join(seq))
  prefix = os.path.join('tmp',prefix)
  print(f"running mmseqs2")
  a3m_lines = cf.run_mmseqs2(seq, prefix, filter=True)
  
  # write a3m to output folder
  with open(f"{output_dir}/msa.a3m","w") as out_a3m:
    out_a3m.write(a3m_lines)

  msa, deletion_matrice = parsers.parse_a3m(a3m_lines)
  
  msas, deletion_matrices = [],[]
  msas.append(msa)
  deletion_matrices.append(deletion_matrice)
  #############################
  # homooligomerize
  #############################
  lengths = [len(seq) for seq in seqs]
  msas_mod, deletion_matrices_mod = cf.homooligomerize_heterooligomer(msas, deletion_matrices,
                                                                    lengths, homooligomers)

  num_res = len(seq)
  feature_dict = {}
  feature_dict.update(pipeline.make_sequence_features(seq, 'test', num_res))
  feature_dict.update(pipeline.make_msa_features(msas_mod, deletion_matrices=deletion_matrices_mod))
  if not use_turbo:
    feature_dict.update(_placeholder_template_feats(0, num_res))


  ################################
  # set chain breaks
  ################################
  Ls = []
  for seq,h in zip(seq.split(":"),homooligomers):
    Ls += [len(s) for s in seq.split("/")] * h
  Ls_plot = sum([[len(seq)]*h for seq,h in zip(seqs,homooligomers)],[])

  
  feature_dict['residue_index'] = cf.chain_break(feature_dict['residue_index'], Ls)
  from string import ascii_uppercase

  with tqdm.notebook.tqdm(total=total, bar_format=TQDM_BAR_FORMAT) as pbar:
    #######################################################################
    # precompile model and recompile only if length changes
    #######################################################################
    if use_turbo:
      name = "model_5_ptm" if use_ptm else "model_5"
      N = len(feature_dict["msa"])
      L = len(feature_dict["residue_index"])
      compiled = (N, L, use_ptm, max_recycles, tol, num_ensemble, max_msa, is_training)
      if "COMPILED" in dir():
        if COMPILED != compiled: recompile = True
      else: recompile = True
      if recompile:
        cf.clear_mem("gpu")
        cfg = config.model_config(name)      
        cfg.data.common.max_extra_msa = min(N,max_extra_msa)
        cfg.data.eval.max_msa_clusters = min(N,max_msa_clusters)
        cfg.data.common.num_recycle = max_recycles
        cfg.model.num_recycle = max_recycles
        cfg.model.recycle_tol = tol
        cfg.data.eval.num_ensemble = num_ensemble

        params = data.get_model_haiku_params(name,'./alphafold/data')
        model_runner = model.RunModel(cfg, params, is_training=is_training)
        COMPILED = compiled
        recompile = False

    else:
      cf.clear_mem("gpu")
      recompile = True

    # cleanup
    if "outs" in dir(): del outs
    outs = {}
    cf.clear_mem("cpu")  

    #######################################################################
    for num, model_name in enumerate(model_names): # for each model
      name = model_name+"_ptm" if use_ptm else model_name

      # setup model and/or params
      params = data.get_model_haiku_params(name, './alphafold/data')
      if use_turbo:
        for k in model_runner.params.keys():
          model_runner.params[k] = params[k]
      else:
        cfg = config.model_config(name)
        cfg.data.common.num_recycle = cfg.model.num_recycle = max_recycles
        cfg.model.recycle_tol = tol
        cfg.data.eval.num_ensemble = num_ensemble
        model_runner = model.RunModel(cfg, params, is_training=is_training)

      for seed in range(num_samples): # for each seed
        # predict
        key = f"{name}_seed_{seed}"
        pbar.set_description(f'Running {key}')      
        processed_feature_dict = model_runner.process_features(feature_dict, random_seed=seed)      
        prediction_result, (r, t) = cf.to(model_runner.predict(processed_feature_dict, random_seed=seed),"cpu")
        outs[key] = parse_results(prediction_result, processed_feature_dict)
        
        # report
        pbar.update(n=1)
        line = f"{key} recycles:{r} tol:{t:.2f} pLDDT:{outs[key]['pLDDT']:.2f}"
        if use_ptm: line += f" pTMscore:{outs[key]['pTMscore']:.2f}"
        print(line)
        if show_images:
          fig = cf.plot_protein(outs[key]["unrelaxed_protein"], Ls=Ls_plot, dpi=100)
          plt.show()
        if save_tmp_pdb:
          tmp_pdb_path = os.path.join(output_dir,f'unranked_{key}_unrelaxed.pdb')
          pdb_lines = protein.to_pdb(outs[key]["unrelaxed_protein"])
          with open(tmp_pdb_path, 'w') as f: f.write(pdb_lines)


        # cleanup
        del processed_feature_dict, prediction_result

      if use_turbo:
        del params
      else:
        del params, model_runner, cfg
        cf.clear_mem("gpu")

    # delete old files
    for f in os.listdir(output_dir):
      if "rank" in f:
        os.remove(os.path.join(output_dir, f))

    # Find the best model according to the mean pLDDT.
    model_rank = list(outs.keys())
    model_rank = [model_rank[i] for i in np.argsort([outs[x][rank_by] for x in model_rank])[::-1]]

    # Write out the prediction
    for n,key in enumerate(model_rank):
      prefix = f"rank_{n+1}_{key}" 
      pred_output_path = os.path.join(output_dir,f'{prefix}_unrelaxed.pdb')
      fig = cf.plot_protein(outs[key]["unrelaxed_protein"], Ls=Ls_plot, dpi=200)
      plt.savefig(os.path.join(output_dir,f'{prefix}.png'), bbox_inches = 'tight')
      plt.close(fig)

      pdb_lines = protein.to_pdb(outs[key]["unrelaxed_protein"])
      with open(pred_output_path, 'w') as f:
        f.write(pdb_lines)
      if use_amber_relax:
        pbar.set_description(f'AMBER relaxation')
        if relax_all or n == 0:
          amber_relaxer = relax.AmberRelaxation(
              max_iterations=0,
              tolerance=2.39,
              stiffness=10.0,
              exclude_residues=[],
              max_outer_iterations=20)
          relaxed_pdb_lines, _, _ = amber_relaxer.process(prot=outs[key]["unrelaxed_protein"])        
          pred_output_path = os.path.join(output_dir,f'{prefix}_relaxed.pdb')
          with open(pred_output_path, 'w') as f:
            f.write(relaxed_pdb_lines)
          pbar.update(n=1)
        
  ############################################################
  print(f"model rank based on {rank_by}")
  for n,key in enumerate(model_rank):
    print(f"rank_{n+1}_{key} {rank_by}:{outs[key][rank_by]:.2f}")
    if use_ptm and save_pae_json:
      pae = outs[key]["pae"]
      max_pae = pae.max()
      # Save pLDDT and predicted aligned error (if it exists)
      pae_output_path = os.path.join(output_dir,f'rank_{n+1}_{key}_pae.json')
      # Save predicted aligned error in the same format as the AF EMBL DB
      rounded_errors = np.round(np.asarray(pae), decimals=1)
      indices = np.indices((len(rounded_errors), len(rounded_errors))) + 1
      indices_1 = indices[0].flatten().tolist()
      indices_2 = indices[1].flatten().tolist()
      pae_data = json.dumps([{
          'residue1': indices_1,
          'residue2': indices_2,
          'distance': rounded_errors.flatten().tolist(),
          'max_predicted_aligned_error': max_pae.item()
      }],
                            indent=None,
                            separators=(',', ':'))
      with open(pae_output_path, 'w') as f:
        f.write(pae_data)
  dpi =  100
  if use_ptm:
    print("predicted alignment error")
    cf.plot_paes([outs[k]["pae"] for k in model_rank], Ls=Ls_plot, dpi=dpi)
    plt.savefig(os.path.join(output_dir,f'predicted_alignment_error.png'), bbox_inches = 'tight', dpi=np.maximum(200,dpi))

  print("predicted contacts")
  cf.plot_adjs([outs[k]["adj"] for k in model_rank], Ls=Ls_plot, dpi=dpi)
  plt.savefig(os.path.join(output_dir,f'predicted_contacts.png'), bbox_inches = 'tight', dpi=np.maximum(200,dpi))

  print("predicted distogram")
  cf.plot_dists([outs[k]["dists"] for k in model_rank], Ls=Ls_plot, dpi=dpi)
  plt.savefig(os.path.join(output_dir,f'predicted_distogram.png'), bbox_inches = 'tight', dpi=np.maximum(200,dpi))

  print("predicted LDDT")
  cf.plot_plddts([outs[k]["plddt"] for k in model_rank], Ls=Ls_plot, dpi=dpi)
  plt.savefig(os.path.join(output_dir,f'predicted_LDDT.png'), bbox_inches = 'tight', dpi=np.maximum(200,dpi))

  %shell zip -FSrj $result_dir"/"$jobname".result.zip" "run.log" $output_dir"/"* 

# Instructions <a name="Instructions"></a>
**Quick start**
1. Change the runtime type to GPU at "Runtime" -> "Change runtime type" (improves speed).
2. Paste your protein sequence in the input field below.
3. Press "Runtime" -> "Run all".
4. The pipeline consists of 10 steps. The currently running steps is indicated by a circle with a stop sign next to it.

**Result zip file contents**

1. PDB formatted structures sorted by avg. pIDDT. (relaxed, unrelaxed).
2. Plots of the model quality.
3. Plots of the MSA coverage.
4. Parameter log file.
5. A3M formatted input MSA.
6. BibTeX file with citations for all used tools and databases.

At the end of the job a download modal box will pop up with a `jobname.result.zip` file. Additionally, if the `save_to_google_drive` option was selected, the `jobname.result.zip` will be uploaded to your Google Drive.

**Using a custom MSA as input**

To predict the structure with a custom MSA (A3M formatted): (1) Change the msa_mode: to "custom", (2) Wait for an upload box to appear at the end of the "Input Protein ..." box. Upload your A3M. The first fasta entry of the A3M must be the query sequence without gaps.

To generate good input MSAs the HHblits server can be used here: https://toolkit.tuebingen.mpg.de/tools/hhblits

After submitting your query, click "Query Template MSA" -> "Download Full A3M". Download the a3m file and upload it to the notebook.

**Troubleshooting**
* Try to restart the session "Runtime" -> "Factory reset runtime".
* Check your input sequence.

**Known issues**
* Colab assigns different types of GPUs with varying amount of memory. Some might have not enough memory to predict the structure.
* Your browser can block the pop-up for downloading the result file. You can choose the `save_to_google_drive` option to upload to Google Drive instead or manually download the result file: Click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).

**Limitations**
* MSAs: MMseqs2 is very precise and sensitive but might find less hits compared to HHblits/HMMer searched against BFD or Mgnify.
* Computing resources: Our MMseqs2 API can probably handle ~20k requests per day.
* For best results, we recommend using the full pipeline: https://github.com/deepmind/alphafold

**Description of the plots**
*   **Number of sequences per position** - We want to see at least 30 sequences per position, for best performance, ideally 100 sequences.
*   **Predicted lDDT per position** - model confidence (out of 100) at each position. Higher the better.
*   **Predicted Alignment Error** - For homooligomers, this could be a useful metric to assess how confident the model is about the interface. Lower the better.



**Bugs**
- If you encounter any bugs, please report the issue to https://github.com/sokrypton/ColabFold/issues


**Acknowledgments**
- We would like to thank the AlphaFold team for developing an excellent model and open sourcing the software. 

- A colab by Sergey Ovchinnikov ([@sokrypton](https://twitter.com/sokrypton)), Milot Mirdita ([@milot_mirdita](https://twitter.com/milot_mirdita)) and Martin Steinegger ([@thesteinegger](https://twitter.com/thesteinegger)).

- Minkyung Baek ([@minkbaek](https://twitter.com/minkbaek)) and Yoshitaka Moriwaki ([@Ag_smith](https://twitter.com/Ag_smith)) for protein-complex prediction proof-of-concept in AlphaFold2.

- Also, credit to [David Koes](https://github.com/dkoes) for his awesome [py3Dmol](https://3dmol.csb.pitt.edu/) plugin, without whom these notebooks would be quite boring!

- For related notebooks see: [ColabFold](https://github.com/sokrypton/ColabFold)
